In [1]:
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,)
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement)



import time
from cmim import CMIMFeatureSelector


import os
import time
import random
import numpy as np
import pandas as pd
import torch
import sys
from rdkit import Chem
from rdkit.Chem import Draw
from matplotlib import pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

from botorch.models.model_list_gp_regression import ModelListGP

from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_mll as fit_gpytorch_model
from botorch.optim.fit import fit_gpytorch_mll_scipy, fit_gpytorch_mll_torch

from botorch.acquisition.analytic import ExpectedImprovement, UpperConfidenceBound
from botorch.models.transforms import Normalize, Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_fully_bayesian_model_nuts
from botorch.models.fully_bayesian import SaasFullyBayesianSingleTaskGP
from gpytorch.kernels import MaternKernel

from botorch.acquisition.analytic import ExpectedImprovement, UpperConfidenceBound, ConstrainedExpectedImprovement
from botorch.acquisition import qExpectedImprovement, qUpperConfidenceBound
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)

import pdb

from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement)

from minepy import MINE
from mordred import Calculator, descriptors

import warnings
warnings.filterwarnings("ignore")

import gpytorch

class MolDAIS:
    
    def __init__(self, problem=None, optimizer_parameters=None, configuration=None, results=None):
        self.problem = problem or MolDAIS.Problem()
        self.optimizer_parameters = optimizer_parameters or MolDAIS.OptimizerParameters()
        self.results = results or MolDAIS.Results()  # Initialize the Results class
        self.configuration = configuration or MolDAIS.Configuration(self.problem, self.optimizer_parameters, self.results)
        self.problemname = self.configuration.set_problemname()  # Automatically set the problemname
        self.iteration = 0

        
        
    class Results:
        
        def __init__(self,X_full=torch.tensor([]), X=torch.tensor([]), y=torch.tensor([]), 
                    best_values=None, best_molecules=None):
            self.X_full = X_full
            self.X = X
            self.y = y
            self.best_values = best_values or []  # To store the best objective value at each iteration
            self.best_molecules = best_molecules or []  # To store the best molecule (SMILES) at each iteration



    class Problem:
        
        def __init__(self, smiles_search_space=None, descriptors_search_space=None, targets=None, experiment_name='None'):
            self.smiles_search_space = smiles_search_space  # pandas series of SMILES
            self.init_smiles_search_space = smiles_search_space  # pandas series of SMILES
            self.descriptors_search_space = descriptors_search_space  # torch.tensor for descriptors
            self.targets = targets  # torch.tensor for targets
            self.init_targets = targets  # torch.tensor for targets
            self.experiment_name = experiment_name

        def compute_descriptors(self):
            calc = Calculator(descriptors, ignore_3D=False)
            # Convert SMILES to RDKit Mol objects
            mols = [Chem.MolFromSmiles(smile) for smile in self.smiles_search_space]
            # Compute Mordred descriptors
            df = calc.pandas(mols,quiet=True,)
            # Remove non-numeric columns and columns with missing values
            df = df.select_dtypes(include=['float64', 'int64']).dropna(axis=1)
            # Convert the descriptors DataFrame to a torch tensor
            self.descriptors_search_space = torch.tensor(df.values, dtype=torch.float32)

            
            
    class OptimizerParameters:
        
        def __init__(self, total_budget=100, initialization_budget=10, batch=1, sparsity_method='MIC',
                     num_sparsity_feats=10, frac_sparsity_feats=0.01, multi_objective=False, constrained=False,
                     acq_fun='EI', use_second_var=False, seed=0, custom_acq_fun=None, custom_sparsity_method=None,
                     custom_model_type=None, custom_fit_strategy=None):
            self.total_budget = total_budget
            self.initialization_budget = initialization_budget
            self.batch = batch
            self.sparsity_method = sparsity_method
            self.num_sparsity_feats = num_sparsity_feats
            self.frac_sparsity_feats = frac_sparsity_feats
            self.multi_objective = multi_objective
            self.constrained = constrained
            self.acq_fun = acq_fun
            self.use_second_var = use_second_var
            self.seed = seed
            self.custom_acq_fun = custom_acq_fun
            self.custom_sparsity_method = custom_sparsity_method
            self.custom_model_type = custom_model_type
            self.custom_fit_strategy = custom_fit_strategy
            if (self.multi_objective or self.constrained):
                self.use_second_var = True
                print('using second variable')

        def set_seed(self, seed):
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)

            
            
    class Configuration:
        
        def __init__(self, problem, optimizer_parameters, results, model=None, sampled_smiles=None, problemname=None, iteration=0):
            self.results = results  # Access results attributes via self.results
            self.model = model
            self.sampled_smiles = sampled_smiles or []
            self.iteration = iteration
            self.problem = problem  # Store the passed problem
            self.optimizer_parameters = optimizer_parameters  # Store optimizer parameters
            self.previous_filename = None
            self.selected_indicies = None

            sparsity_method = self.optimizer_parameters.sparsity_method
            if sparsity_method == 'SAAS':
                self.model_type = 'SAAS'
            else:
                self.model_type = self.optimizer_parameters.custom_model_type or 'GP'  # Default model type to GP

        def set_problemname(self):
            """Set the problem name for saving and loading."""
            # Build a descriptive filename that includes the problem name, sparsity method, model type, acquisition function, and iteration
            problem_name = self.problem.experiment_name or 'UnnamedProblem'
            sparsity_method = self.optimizer_parameters.sparsity_method
            model_type = self.model_type
            acq_fun = self.optimizer_parameters.acq_fun
            iteration_str = f"iter_{self.iteration}"
            seed = self.optimizer_parameters.seed
            num_feats = self.optimizer_parameters.num_sparsity_feats 
            # Generate descriptive filename
            filename = f"./results/New_{problem_name}_sparsity-{sparsity_method}{num_feats}_model-{model_type}_acq-{acq_fun}_{iteration_str}_seed-{seed}.pkl"
            return filename

        def apply_sparsity(self):
            """Apply sparsity to the training data."""
            sampled_descriptors = self.results.X_full  # Access X_full from results
            self.results.X = torch.zeros((self.results.y.shape[-1], sampled_descriptors.shape[0], self.optimizer_parameters.num_sparsity_feats))  # Update X in results
            self.selected_indicies = torch.zeros((self.results.y.shape[-1], self.optimizer_parameters.num_sparsity_feats))
            
            if self.optimizer_parameters.sparsity_method == 'Spearman':
                for j in range(self.results.y.shape[-1]):
                    correlations = []
                    for i in range(sampled_descriptors.shape[1]):
                        try:
                            corr, _ = torch.corrcoef(torch.stack([sampled_descriptors[:, i], self.results.y[:, j].squeeze()]))[0, 1]
                        except:
                            corr = 0
                        correlations.append(corr)

                    correlations = torch.abs(torch.tensor(correlations))
                    top_indices = torch.argsort(correlations, descending=True)[:self.optimizer_parameters.num_sparsity_feats]

                    self.results.X[j, :, :] = sampled_descriptors[:, top_indices].unsqueeze(0)
                    self.selected_indicies[j,:] =top_indices.to(torch.int)

            elif self.optimizer_parameters.sparsity_method == 'MIC':
                mine = MINE(alpha=0.6, c=20)
                for j in range(self.results.y.shape[-1]):
                    mic_scores = []
                    for i in range(sampled_descriptors.shape[1]):
                        feature = sampled_descriptors[:, i].numpy()
                        target = self.results.y[:, j].squeeze().numpy()
                        mine.compute_score(feature, target)
                        mic_scores.append(mine.mic())

                    mic_scores = torch.tensor(mic_scores)
                    top_indices = torch.argsort(mic_scores, descending=True)[:self.optimizer_parameters.num_sparsity_feats]

                    self.results.X[j, :, :] = sampled_descriptors[:, top_indices].unsqueeze(0)
                    self.selected_indicies[j,:] =top_indices.to(torch.int)

                    
                    
            elif self.optimizer_parameters.sparsity_method == 'MI':
                self.selected_indicies = torch.zeros((self.results.y.shape[-1], self.optimizer_parameters.num_sparsity_feats))

                for j in range(self.results.y.shape[-1]):

                    mi_scores = mutual_info_regression(sampled_descriptors.numpy(), self.results.y[:, j].numpy())
                    k = self.optimizer_parameters.num_sparsity_feats
                    top_indices = np.argsort(arr)[-k:][::-1]

                    self.selected_indicies[j, :] = torch.tensor(top_indices)
                    self.results.X[j, :, :] = sampled_descriptors[:, top_indices].unsqueeze(0)


            elif self.optimizer_parameters.sparsity_method == 'CMI':
                for j in range(self.results.y.shape[-1]):
                    selector = CMIMFeatureSelector(n_features_to_select=self.optimizer_parameters.num_sparsity_feats,task='regression')
                    selector.fit(sampled_descriptors.numpy(), self.results.y[:, j].numpy())
                    X_transformed = selector.transform(sampled_descriptors)                                    
                    try: self.results.X[j,:,:] = torch.tensor(X_transformed)
                    except: pdb.set_trace()

            elif self.optimizer_parameters.sparsity_method == 'SAAS':
                # Use the SAAS method for sparsity
                self.results.X = torch.zeros((self.results.y.shape[-1], sampled_descriptors.shape[0], sampled_descriptors.shape[1]))  # Update X in results
                self.selected_indicies = torch.zeros((self.results.y.shape[-1],sampled_descriptors.shape[-1] ))
            
                for j in range(self.results.y.shape[-1]):
                  self.results.X[j,:,:] = sampled_descriptors
                  self.selected_indicies[j,:] = torch.arange(sampled_descriptors.shape[-1]).to(torch.int)

                    
                    
            return self.results.X

        def create_model(self):
            """Create and return a model depending on the selected type: GP or SAAS-GP."""
            model_type = self.model_type  # Use the provided model type or default to 'GP'

            if self.optimizer_parameters.use_second_var:
                if model_type == 'GP':
                    model = []
                    mll = []
                    for i in range(self.results.y.shape[-1]):
                        model.append(SingleTaskGP(train_X=self.results.X[i,:, :], 
                                                  train_Y=self.results.y[:, i].unsqueeze(1),
                                                  input_transform=Normalize(d=self.results.X.size(-1)), 
                                                  outcome_transform=Standardize(m=1), 
                                                  covar_module=MaternKernel(nu=2.5, ard_num_dims=self.results.X.shape[-1])  # Using Matern 2.5 kernel with ARD
                                                  ))
                        mll.append(ExactMarginalLogLikelihood(likelihood=model[i].likelihood, model=model[i]))
                    self.model = model
                    self.mll = mll
                elif model_type == 'SAAS':
                    model = []
                    for i in range(self.results.y.shape[-1]):
                        model.append(SaasFullyBayesianSingleTaskGP(train_X=self.results.X[i, :, :], train_Y=self.results.y[:, i].unsqueeze(1),
                                                                  input_transform=Normalize(d=self.results.X.size(-1)), outcome_transform=Standardize(m=1)))
                    self.model = model
                    self.mll = None
            else:
                if model_type == 'GP':
                    # Standard GP model
                    self.model = SingleTaskGP(
                        train_X=self.results.X[0, :, :],  # Access X from results
                        train_Y=self.results.y,  # Access y from results
                        input_transform=Normalize(d=self.results.X.shape[-1]),  # Normalize input features
                        outcome_transform=Standardize(m=1),   # Standardize the outputs
                        covar_module=MaternKernel(nu=2.5, ard_num_dims=self.results.X.shape[-1])  # Using Matern 2.5 kernel with ARD

                    )
                    self.mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)

                elif model_type == 'SAAS':
                    # SAAS-GP model
                    self.model = SaasFullyBayesianSingleTaskGP(
                        train_X=self.results.X[0, :, :],  # Access X from results
                        train_Y=self.results.y,  # Access y from results
                        input_transform=Normalize(d=self.results.X.shape[-1]),  # Normalize input features
                        outcome_transform=Standardize(m=1)  # Standardize the outputs
                    )
                    self.mll = None  # SAAS-GP uses a fully Bayesian fitting approach, no marginal likelihood
                else:
                    raise ValueError(f"Unknown model type: {model_type}")

        def fit_model(self):
            """Fit the model based on its type."""
            model_type = self.model_type  # Use the provided model type or default to 'GP'

            if model_type == 'GP':
                if not self.optimizer_parameters.use_second_var:
                    try:
                        with gpytorch.settings.cholesky_max_tries(100):
                            fit_gpytorch_model(self.mll)
                    except:
                        try: 
                            fit_gpytorch_mll_scipy(self.mll) 
                        except:
                            fit_gpytorch_mll_torch(self.mll)
                    
                else:
                    for mll_i in self.mll:
                        fit_gpytorch_model(mll_i)
                    self.model = ModelListGP(*self.model)

            elif model_type == 'SAAS':
                warmup_steps = 12
                num_samples = 25
                thinning = 16              
                if not self.optimizer_parameters.use_second_var:
                    fit_fully_bayesian_model_nuts(
                        self.model,
                        warmup_steps=warmup_steps,  # Number of warmup steps
                        num_samples=num_samples,   # Number of MCMC samples
                        thinning=thinning  # Thinning rate
                    )
                    self.model = self.model
                else:
                    for model_i in self.model:
                        fit_fully_bayesian_model_nuts(model_i, warmup_steps=warmup_steps, num_samples=num_samples, thinning=thinning, disable_progbar=False)
                    self.model = ModelListGP(*self.model)
            else:
                raise ValueError(f"Unknown model type: {model_type}")

        def get_sample(self):
            """Get the next sample based on the selected acquisition function."""
            X_space = self.problem.descriptors_search_space[:, self.selected_indicies.to(torch.int)].unsqueeze(1).to(torch.float)
            if self.optimizer_parameters.use_second_var:
                dim = self.results.X.shape[1]  # Access X from results
                if self.optimizer_parameters.multi_objective:
                    try:
                        partitioning = partitioning = FastNondominatedPartitioning(ref_point=torch.max(self.results.y, dim=0).values,Y=self.results.y)
                    except:
                        pdb.set_trace()
                    try: del f_acq
                    except: pass
                    f_acq = qExpectedHypervolumeImprovement(
                        model=self.model,
                        ref_point=torch.max(self.results.y[:,0]*self.results.y[:,1], dim=0),
                        partitioning=partitioning, )

                    acq = safe_eval(f_acq, X_space) 
                    max_acq = torch.max(acq)
                    max_acq_index = torch.argmax(acq)
                    next_sample_index = max_acq_index

                else:  # constrained
                    lwr_bound = 0
                    feas_vals = self.results.y[:, 1] <= lwr_bound  # Access y from results
                    try:
                        max_val = (self.results.y[:, 0] * feas_vals).max()  # Access y from results
                    except:
                        pdb.set_trace()
                    constraints = {1: (lwr_bound, None)}
                    f_acq = ConstrainedExpectedImprovement(self.model, max_val, 0, constraints)

                    acq = safe_eval(f_acq, X_space)  # Access X from results
                    max_acq = torch.max(acq)
                    max_acq_index = torch.argmax(acq)
                    next_sample_index = max_acq_index

            else: # single variable
                if self.optimizer_parameters.acq_fun == 'EI':
                    acq_function = ExpectedImprovement(self.model, best_f=torch.max(self.results.y))  # Access y from results
                elif self.optimizer_parameters.acq_fun == 'UCB':
                    acq_function = UpperConfidenceBound(self.model, beta=2.0)
                else:
                    raise ValueError("Unknown acquisition function")

                with torch.no_grad():
                    try: acq_vals = acq_function(X_space)  # Access X from results
                    except: pdb.set_trace()
                    next_sample_index = torch.argmax(acq_vals).item()
                    
            return next_sample_index





        def save_iteration(self):
            """Save the current state of optimization."""
            iteration_data = {
                'X_full': self.results.X_full,
                'X': self.results.X,
                'y': self.results.y,
                'model': self.model,
                'best_values': self.results.best_values,
                'best_molecules': self.results.best_molecules,
                'iteration': self.iteration,
                'targets': self.problem.targets,
                'sampled_smiles': self.sampled_smiles
            }
            filename = self.set_problemname()
            if self.previous_filename is not None and os.path.exists(self.previous_filename):
                os.remove(self.previous_filename)
            
            with open(filename, 'wb') as f:
                torch.save(iteration_data, f)
                
            self.previous_filename = filename
            
            print(f"Iteration {self.iteration} saved as {filename}")

        def load_from_checkpoint(self, checkpoint_filename):
            """Load the state of optimization from a saved checkpoint."""
            with open(checkpoint_filename, 'rb') as f:
                checkpoint_data = torch.load(f)
            self.results.X_full = checkpoint_data['X_full']
            self.results.X = checkpoint_data['X']
            self.results.y = checkpoint_data['y']
            self.model = checkpoint_data['model']
            self.results.best_values = checkpoint_data['best_values']
            self.results.best_molecules = checkpoint_data['best_molecules']
            self.iteration = checkpoint_data['iteration']
            self.problem.targets = checkpoint_data['targets']
            self.sampled_smiles = checkpoint_data['sampled_smiles']
            print(f"Loaded checkpoint from {checkpoint_filename}")


        def get_max_y(self, y, all=False):
          if self.optimizer_parameters.multi_objective:
            y_ = torch.prod(y, dim=1)
            return max(y_) if not all else y_

          elif self.optimizer_parameters.constrained:
            feas = self.results.y[:,1]<=0
            return max(self.results.y[:,0]*feas)  if not all else self.results.y[:,0]*feas
          else:
            return max(self.results.y) if not all else self.results.y



        def optimize(self):
            """Run the optimization loop."""
            # Set the seed for reproducibility
            seed = self.optimizer_parameters.seed

            total_budget = self.optimizer_parameters.total_budget
            init_budget = self.optimizer_parameters.initialization_budget
            

            # Check if training data is empty
            if self.results.X.numel() == 0:  # Check if X (training data) is empty
                print(f"Initializing with random {init_budget} samples from the search space.")

                # Randomly sample from the search space using initialization budget
                np.random.seed(seed)
                init_indices = np.random.choice(np.arange(len(self.problem.smiles_search_space)), init_budget, replace=False)

                # Add sampled SMILES and corresponding descriptors/targets to the training data
                sampled_smiles = [self.problem.smiles_search_space[idx] for idx in init_indices]
                self.results.X_full = self.problem.descriptors_search_space[init_indices]
                self.results.y = self.problem.targets[init_indices]
                self.results.X = self.apply_sparsity()

                # Remove the sampled data from the search space and targets
                self.problem.smiles_search_space = [smile for i, smile in enumerate(self.problem.smiles_search_space) if i not in init_indices]
                self.problem.descriptors_search_space = torch.cat([self.problem.descriptors_search_space[i].unsqueeze(0) for i in range(len(self.problem.descriptors_search_space)) if i not in init_indices])
                self.problem.targets = torch.cat([self.problem.targets[i].unsqueeze(0) for i in range(len(self.problem.targets)) if i not in init_indices])

                # Store the sampled SMILES for reference
                self.sampled_smiles.extend(sampled_smiles)

            best_value = self.get_max_y(self.results.y).item()
            best_smiles = self.sampled_smiles[torch.argmax(self.get_max_y(self.results.y, all=True))]

            for bo_iter in range(self.iteration, total_budget - init_budget):
                self.iteration = bo_iter
                print(f"Starting BO Iteration: {bo_iter + init_budget} / {total_budget}")
                self.create_model()
                self.fit_model()

                # Get the next sample
                next_sample_idx = self.get_sample()

                next_sample_value = self.problem.targets[next_sample_idx, :]
                
                
                if self.optimizer_parameters.use_second_var:
                    next_score = self.get_max_y(next_sample_value.unsqueeze(0))
                else:
                    next_score = next_sample_value

                # Track the best objective value and the corresponding molecule
                if next_score > best_value:
                    best_value = next_score
                    best_smiles = self.problem.smiles_search_space[next_sample_idx]
                try:
                    best_value = best_value.item()
                except:
                    pass
                self.results.best_values.append(best_value)
                self.results.best_molecules.append(best_smiles)

                # Update training data (X, y) with the new sample
                new_X_sample = self.problem.descriptors_search_space[next_sample_idx].unsqueeze(0)
                new_y_sample = self.problem.targets[next_sample_idx].unsqueeze(0)
                self.results.X_full = torch.cat((self.results.X_full, new_X_sample), dim=0)
                self.results.y = torch.cat((self.results.y, new_y_sample), dim=0)
                self.apply_sparsity()

                # Remove the selected sample from the search space and targets
                self.problem.smiles_search_space.pop(next_sample_idx)
                self.problem.descriptors_search_space = torch.cat([self.problem.descriptors_search_space[i].unsqueeze(0) for i in range(len(self.problem.descriptors_search_space)) if i != next_sample_idx], dim=0)
                self.problem.targets = torch.cat([self.problem.targets[i].unsqueeze(0) for i in range(len(self.problem.targets)) if i != next_sample_idx], dim=0)

                # Store the sampled SMILES for reference
                self.sampled_smiles.append(best_smiles)

                self.iteration = bo_iter + 1
                
                self.save_iteration()


        def plot_convergence(self):
            """Plot the best objective value at each iteration and display the best molecules."""
            zo = 0
            rd_opts = Draw.DrawingOptions()
            rd_opts.bgColor=None

            
            fig, ax = plt.subplots(figsize=(8,5))

            # Plot the convergence curve (best objective values)
            ax.plot(range(len(self.results.best_values)), self.results.best_values, marker='o', label='Best Objective Value')
            #ax.set_title('Convergence Plot of Best Objective Value')
            ax.set_xlabel('Iteration')
            ax.set_ylabel('Best Objective Value')
            ax.grid(True)

            top_row = self.get_max_y(self.problem.init_targets)
            bottom_row = min(self.get_max_y(self.problem.init_targets, all =True))
           
            plt.plot([0,len(self.results.best_values)], [top_row, top_row],'k:', label='Dataset min/max') 
            plt.plot([0,len(self.results.best_values)], [bottom_row, bottom_row],'k:') 

            # Draw molecules on the plot at various points --incomplete
            # - does not look good in many cases 
            # - consider changing frequency and size of molecule images
            if 0:
              value_ = -1E12
              for i, (value, smiles) in enumerate(zip(self.results.best_values, self.results.best_molecules)):
                  if value>value_:  
                      mol = Chem.MolFromSmiles(smiles)
                      if mol:
                          img = Draw.MolToImage(mol,options=rd_opts)
                          # Display the molecule image on the plot
                          imgbox = OffsetImage(img, zoom=0.25)
                          ab = AnnotationBbox(imgbox, (i, top_row), frameon=False, zorder=zo)
                          ax.add_artist(ab)
                  value_=value
            
            ax.legend()
            plt.tight_layout()
            plt.ylim(bottom_row*.9,top_row*1.1)
            plt.xlim(0,len(self.results.best_values)-1)
            plt.show()

            

##########################################################################

# Helper function to update the test/train data
def update_test_train_data(next_sample_idx, smiles_search_space, descriptors_search_space):
    new_smiles_space = smiles_search_space[:next_sample_idx] + smiles_search_space[next_sample_idx + 1:]
    new_descriptors = torch.cat((descriptors_search_space[:next_sample_idx], descriptors_search_space[next_sample_idx + 1:]), dim=0)
    return new_smiles_space, new_descriptors


def safe_eval(f, x):
  n = len(x)
  nm = 10
  splits = int(np.ceil(n/nm))
  obs = []
  for i in range(splits):
    if i+1 != splits:
      xi = x[nm*i:nm*(i+1)].detach()
    else:
      xi = x[nm*i:].detach()
    try:obs.append(f(xi).detach().tolist())
    except: pdb.set_trace()
  obs = torch.tensor(sum(obs, []))
  return obs


from sklearn.feature_selection import mutual_info_regression



                
                


In [2]:

name = 'dG'; data_file = 'MORDRED_SMILES_dft_Gsolv.csv'
df_dg = pd.read_csv('../../prop_data/'+data_file)#.drop(columns=['Unnamed: 0'])


name = 'E'; data_file = 'MORDRED_SMILES_dft_E.csv'
df_e0 = pd.read_csv('../../prop_data/'+data_file)#.drop(columns=['Unnamed: 0'])



from rdkit import Chem
from rdkit.Chem import Descriptors
import numpy as np

def compute_molecular_weights(smiles_list):
    molecular_weights = []
    for smiles in smiles_list:
        molecule = Chem.MolFromSmiles(smiles)
        if molecule:
            mw = Descriptors.MolWt(molecule)
            molecular_weights.append(mw)
        else:
            molecular_weights.append(np.nan)  # Use NaN for invalid SMILES
    return np.array(molecular_weights)

smiles_list = df_dg['SMILES'].to_list()
molecular_weights = compute_molecular_weights(smiles_list)


# Step 1: load data
smiles_list = df_dg['SMILES'].to_list()
values1 = -1*torch.tensor(df_dg['Gsolv'].values, dtype=torch.float32).reshape(-1,1)
F = 96485
values2 = torch.tensor(((df_e0['E'].values+0.76)*2*F)/ (3.600*molecular_weights), dtype=torch.float32).reshape(-1,1)



values = torch.cat([values1,values2], dim=1)




In [3]:
df_dg.columns[:4], df_e0.columns[:4]

(Index(['SMILES', 'Gsolv', 'nAcid', 'nBase'], dtype='object'),
 Index(['SMILES', 'E', 'ABC', 'ABCGG'], dtype='object'))

In [4]:

# Step 1: load data
smiles_list = df_dg['SMILES'].to_list()
values1 = -1*torch.tensor(df_dg['Gsolv'].values, dtype=torch.float32).reshape(-1,1)
F = 96485
values2 = torch.tensor(((df_e0['E'].values+0.76)*2*F)/ (3.600*molecular_weights), dtype=torch.float32).reshape(-1,1)



values = torch.cat([values1,values2], dim=1)


descriptors_search_space = torch.tensor(df_dg.iloc[:,2:].select_dtypes(include=['float64', 'int64']).dropna(axis=1).values)
                          



In [5]:

# Step 2: Define the problem
PS_problem = MolDAIS.Problem(smiles_search_space=smiles_list,
                          targets=values,
                          descriptors_search_space = descriptors_search_space,
                          experiment_name=name)
#PS_problem.compute_descriptors()


r_list = []
for kk in range(10):
    # Step 3: Define optimizer parameters
    optimizer_parameters = MolDAIS.OptimizerParameters(
        sparsity_method='CMI',  # Use MIC as the sparsity method
        acq_fun='EI',  # Use Expected Improvement (EI) as the acquisition function
        num_sparsity_feats=10,  # Let's use 10 features for sparsity
        total_budget=100,  # Number of iterations to run
        initialization_budget=10,  # Number of initial points
        seed=kk,  # Seed for reproducibility
        multi_objective = True,
    )

    # Step 4: Create the optimization problem and the MolDAIS object
    mol_dais = MolDAIS(problem=PS_problem, optimizer_parameters=optimizer_parameters)
    
    #checkpoint_filename = f'./results/MOO_E_sparsity-CMI10_model-GP_acq-EI_iter_40_seed-{kk}.pkl'
    #mol_dais.configuration.load_from_checkpoint(checkpoint_filename)
    
    # Step 5: Run the optimization for a few iterations
    print("\n Running optimization for the first time:")
    mol_dais.configuration.optimize()





using second variable

 Running optimization for the first time:
Initializing with random 10 samples from the search space.
Starting BO Iteration: 10 / 100
Iteration 1 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_1_seed-0.pkl
Starting BO Iteration: 11 / 100
Iteration 2 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_2_seed-0.pkl
Starting BO Iteration: 12 / 100
Iteration 3 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_3_seed-0.pkl
Starting BO Iteration: 13 / 100
Iteration 4 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_4_seed-0.pkl
Starting BO Iteration: 14 / 100
Iteration 5 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_5_seed-0.pkl
Starting BO Iteration: 15 / 100
Iteration 6 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_6_seed-0.pkl
Starting BO Iteration: 16 / 100
Iteration 7 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_7_seed-0.pkl
Starting BO Iteration: 17 / 100
Iteration 8 saved 

Iteration 69 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_69_seed-0.pkl
Starting BO Iteration: 79 / 100
Iteration 70 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_70_seed-0.pkl
Starting BO Iteration: 80 / 100
Iteration 71 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_71_seed-0.pkl
Starting BO Iteration: 81 / 100
Iteration 72 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_72_seed-0.pkl
Starting BO Iteration: 82 / 100
Iteration 73 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_73_seed-0.pkl
Starting BO Iteration: 83 / 100
Iteration 74 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_74_seed-0.pkl
Starting BO Iteration: 84 / 100
Iteration 75 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_75_seed-0.pkl
Starting BO Iteration: 85 / 100
Iteration 76 saved as ./results/New_E_sparsity-CMI10_model-GP_acq-EI_iter_76_seed-0.pkl
Starting BO Iteration: 86 / 100
Iteration 77 saved as ./results/New_E_sp

BdbQuit: 

In [78]:
# random sample

# Step 2: Define the problem
PS_problem = MolDAIS.Problem(smiles_search_space=smiles_list,
                          targets=values,
                          descriptors_search_space = descriptors_search_space,
                          experiment_name=name)
#PS_problem.compute_descriptors()


r_list = []
for kk in range(10):
    # Step 3: Define optimizer parameters
    optimizer_parameters = MolDAIS.OptimizerParameters(
        sparsity_method='MI',  # Use MIC as the sparsity method
        acq_fun='Random',  # Use Expected Improvement (EI) as the acquisition function
        num_sparsity_feats=10,  # Let's use 10 features for sparsity
        total_budget=100,  # Number of iterations to run
        initialization_budget=10,  # Number of initial points
        seed=kk,  # Seed for reproducibility
        multi_objective = True,
    )

    # Step 4: Create the optimization problem and the MolDAIS object
    mol_dais = MolDAIS(problem=PS_problem, optimizer_parameters=optimizer_parameters)
    
    # Step 5: Run the optimization for a few iterations
    print("\n Running optimization for the first time:")
    mol_dais.configuration.optimize()
    mol_dais.configuration.save_iteration()

using second variable

 Running optimization for the first time:
Initializing with random 100 samples from the search space.
Iteration 0 saved as ./results/MOO_E_sparsity-MI10_model-GP_acq-Random_iter_0_seed-0.pkl
using second variable

 Running optimization for the first time:
Initializing with random 100 samples from the search space.
Iteration 0 saved as ./results/MOO_E_sparsity-MI10_model-GP_acq-Random_iter_0_seed-1.pkl
using second variable

 Running optimization for the first time:
Initializing with random 100 samples from the search space.
Iteration 0 saved as ./results/MOO_E_sparsity-MI10_model-GP_acq-Random_iter_0_seed-2.pkl
using second variable

 Running optimization for the first time:
Initializing with random 100 samples from the search space.
Iteration 0 saved as ./results/MOO_E_sparsity-MI10_model-GP_acq-Random_iter_0_seed-3.pkl
using second variable

 Running optimization for the first time:
Initializing with random 100 samples from the search space.
Iteration 0 saved a

In [21]:
Y = values
hv = Y[:,1]*Y[:,0]

plt.figure(figsize=(12,10))
sns.kdeplot(hv,cut=0, linewidth=4)
plt.xlabel(r"$-\Delta G \cdot E^o$")
plt.tight_layout()
plt.savefig('./figs/MOO_density.png')


torch.Size([50, 2])

In [ ]:

moldais_moo = []
sgp_moo  = []
rnd_moo  = []
tmfp_moo = []


for seed in range(10):
    moldais_moo.append( torch.load(f'./mic_res/results/MOO_E_sparsity-CMI10_model-GP_acq-EI_iter_40_seed-{seed}.pkl')['best_values'])
moldais = np.array(moldais_moo)
moldais_m = moldais.mean(axis=0)
moldais_s = moldais.std(axis=0)    
    
    
    
#for seed in range(10):    
#    sgp_moo.append(torch.load(f'./mic_res/results/MOO_E_sparsity-MI10000_model-GP_acq-EI_iter_40_seed-{seed}.pkl')['best_values'])
#sgp = np.array(sgp_moo)
#sgp_m = sgp.mean(axis=0)
#sgp_s = sgp.std(axis=0)        



for seed in range(10):    
    Yi = torch.load(f'./mic_res/results/MOO_E_sparsity-MI10_model-GP_acq-Random_iter_0_seed-{seed}.pkl')['y']
    Y = Yi.prod(1)
    ybest = [Y[:k].max().item() for k in range(10,50)]
    rnd_moo.append(ybest)
rnd = np.array(rnd_moo)
rnd_m = rnd.mean(axis=0)
rnd_s = rnd.std(axis=0)        

    

num_list = []
df_g = pd.read_csv('./mic_res/results/Gauche_MOO.csv')['y1']
for i in range(10):
    str_list = df_g[i].replace('\n','').replace('tensor([','').replace('Tensor([','').replace('       ','').replace('],dtype=torch.float64)','').replace('], dtype=torch.float64)','').replace(')','').split(', ')
    num_list.append( np.array([float(x.replace(']', '')) for x in str_list]) )
gauche = np.array(num_list)[:,:40]
gauche_m = gauche.mean(axis=0)
gauche_s = gauche.std(axis=0)

In [ ]:

plt.plot([0,41], [hv.max(),hv.max()], 'k:'  )

plt.plot(np.arange(1,41), moldais_m, label='MolDAIS')
plt.fill_between(np.arange(1,41),moldais_m+1.95*moldais_s/10**0.5,moldais_m-1.95*moldais_s/10**0.5, alpha=.3)

#plt.plot(np.arange(1,41), sgp_m, label='SGP')
#plt.fill_between(np.arange(1,41),sgp_m+1.95*sgp_s/10**0.5,sgp_m-1.95*sgp_s/10**0.5, alpha=.3)

plt.plot(np.arange(1,41), rnd_m, label='Random')
plt.fill_between(np.arange(1,41),rnd_m+1.95*rnd_s/10**0.5,rnd_m-1.95*rnd_s/10**0.5, alpha=.3)


plt.plot(np.arange(1,41),gauche_m, label='TM-FP')
plt.fill_between(np.arange(1,41),gauche_m+1.95*gauche_s/10**0.5,gauche_m-1.95*gauche_s/10**0.5, alpha=.3)



plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))

#plt.yscale('exp')
#plt.ylim(0)
plt.xlim(1,40)
plt.legend(fontsize=14, ncols = 1, )
plt.ylabel(r"$-\Delta G \cdot E^o$")




In [ ]:
import pandas as pd
import seaborn as sns
sorted_results5 = Res_dict_raw['random']


seeds = [0,1,2,3,4,5,6,7,8,9]

sorted_results1 = Res_dict_raw['SAAS']
sorted_results2 = Res_dict_raw['GP_full']
sorted_results3 = Res_dict_raw['GP_PCA']
sorted_results4 = Res_dict_raw['Linear']

df2 = pd.DataFrame()

col_name = r'Quantile scores of $m^* $'

end = -1

d =  pd.DataFrame({'Method':['MolDAIS']*len(seeds), col_name:CQ(moldais[:,end],hv)})
df2 = pd.concat([df2, d])

#d =  pd.DataFrame({'Method':['SGP']*len(seeds), col_name:CQ(sgp[:,end],hv)})
#df2 = pd.concat([df2, d])


d =  pd.DataFrame({'Method':['Random']*len(seeds), col_name:CQ(rnd[:,end],hv)})
df2 = pd.concat([df2, d])



d =  pd.DataFrame({'Method':['TM-FP']*len(seeds), col_name:CQ(gauche[:,end],hv)})
df2 = pd.concat([df2, d])


C = [  'tab:cyan',
 'tab:orange',
 'tab:green',
 'tab:red',
 'tab:purple',

         ]

plt.figure(figsize=(13,9))
for i,ls in enumerate(range(4)):
  plt.axvline(i, color=C[i], linestyle='-', zorder=0, linewidth=184, alpha=.15)
#plt.axhline(1,color='k', linestyle=':',zorder=0)
#plt.axhline(.99,color='k', linestyle=':',zorder=0)
#plt.axhline(.98,color='k', linestyle=':',zorder=0)
plt.axhline(1,color='k', linestyle=':',zorder=0)

sns.violinplot(data=df2, x='Method', y=col_name, cut=0, inner="box", scale='width', palette=C, linewidth=3,linecolor='k')
#plt.title('Lipophilicity')
plt.xlabel(None)
#plt.ylim(.93,1.001)
#density_norm{“area”, “count”, “width”}
plt.xticks(rotation=0)

plt.tight_layout()
plt.savefig('./figs/MOO_finalVals.png')

